In [ ]:
# default_exp signal_period_and_magnitude_estimator

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export 
import numpy as np

In [ ]:
#export
class SignalPeriodAndMagnitudeEstimator:
    def __init__(self,fs = 100., cool_down_zero_crossing_in_s = 0.1, peak_finding_delay_in_s = 0.1, lpf_alpha = 0.3):
        self.cool_down_zero_crossing_in_s = cool_down_zero_crossing_in_s
        self.peak_finding_delay_in_s = peak_finding_delay_in_s
        self.previous_polarity = None
        self.cool_down_in_progress = False
        self.cool_down_t0 = None
        
        self.search_for_positive_peak = False
        self.search_for_negative_peak = False
        self.positive_peak_found = False
        self.negative_peak_found = False
        self.pos_peak_t0 = None
        self.pos_peak_trigger_t = None
        self.pos_peak_amp = None
        self.neg_peak_t0 = None
        self.neg_peak_trigger_t = None
        self.neg_peak_amp = None
        self.lpf_alpha = lpf_alpha
        self.respiration_period_est = 0.
        self.respiration_magnitude_est = 0.
        
        self.ready = False
        
    def filter_samples(self,t_in,s_in):
        n = len(t_in)
        s_out = np.zeros((6,n))
        if(n > 0):
            for i in range(n):
                t = t_in[i]
                s = s_in[0,i]
                if(not self.ready): #initialization
                    #s_out[0,i] = 0.
                    self.previous_polarity = np.sign(s_in[0,0])
                    if(self.previous_polarity == 0.):
                        self.previous_polarity = 1.
                    self.ready = True
                else: #initialized           
                    if(self.cool_down_in_progress):
                        if(t - self.cool_down_t0 >= self.cool_down_zero_crossing_in_s):
                            #s_out[0,i] = 0.
                            self.cool_down_in_progress = False
                        else:
                            s_out[0,i] = 1.
                    else:
                        s_sign = np.sign(s)
                        if(s_sign == 0.):
                            s_sign = 1.
                        if(s_sign != self.previous_polarity): #zero-crossing
                            self.previous_polarity = s_sign
                            self.cool_down_in_progress = True
                            self.cool_down_t0 = t
                            s_out[0,i] = 1.
                            if(s_sign > 0.):
                                self.search_for_positive_peak = True
                                self.search_for_negative_peak = False
                                self.pos_peak_t0 = t
                                self.pos_peak_amp = s
                            else:
                                self.search_for_positive_peak = False
                                self.search_for_negative_peak = True
                                self.neg_peak_t0 = t
                                self.neg_peak_amp = s
                if(self.search_for_positive_peak):
                    if(t - self.pos_peak_t0 < self.peak_finding_delay_in_s):
                        if(s > self.pos_peak_amp):
                            self.pos_peak_t0 = t
                            self.pos_peak_amp = s
                    else:
                        self.search_for_positive_peak = False
                        self.positive_peak_found = True
                        
                        if(self.pos_peak_trigger_t is not None):
                            self.respiration_period_est = (1. - self.lpf_alpha) * self.respiration_period_est + self.lpf_alpha * (t - self.pos_peak_trigger_t)
                            self.respiration_magnitude_est = (1. - self.lpf_alpha) * self.respiration_magnitude_est + self.lpf_alpha * (self.pos_peak_amp - self.neg_peak_amp)
                        
                        self.pos_peak_trigger_t = t
                        self.pos_peak_amp = s
                if(self.search_for_negative_peak):
                    if(t - self.neg_peak_t0 < self.peak_finding_delay_in_s):
                        if(s < self.neg_peak_amp):
                            self.neg_peak_t0 = t
                            self.neg_peak_amp = s
                    else:
                        self.search_for_negative_peak = False
                        self.negative_peak_found = True
                        self.neg_peak_trigger_t = t
                        self.neg_peak_amp = s
                if(self.positive_peak_found):
                    if(t - self.pos_peak_trigger_t < self.peak_finding_delay_in_s):
                        s_out[2,i] = 1.
                    else:
                        self.positive_peak_found = False
                if(self.negative_peak_found):
                    if(t - self.neg_peak_trigger_t < self.peak_finding_delay_in_s):
                        s_out[3,i] = 1.
                    else:
                        self.negative_peak_found = False
                
                s_out[1,i] = self.previous_polarity #+1 for positive motion, -1 for negative motion
                s_out[4,i] = self.respiration_period_est
                s_out[5,i] = self.respiration_magnitude_est
            return s_out
        else:
            return np.array()

In [ ]:
# say_hello("Sylvain")

'Hello Sylvain!'

In [ ]:
# assert say_hello("Jeremy")=="Hello Jeremy!"